## Installing & Importing our Prequisites

In [1]:
# Installing my prequisites
%pip install pdfplumber

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Importing Libraries
import pdfplumber # Assist in reading pdf files
import os # Assist in navigating to folders & files
import re # Helps in searching and extracting specific pieces of text
import pandas as pd

## Loading our Data

In [3]:
folder_path = r"C:\Users\USER\Desktop\SQL and Excel Files\Test-20250930T093447Z-1-001\Test\Invoices"

# Creating a list to store all pdf files
pdf_files = [
    f for f in os.listdir(folder_path) if f.endswith('.pdf')
]
print("PDF files found:", pdf_files)

PDF files found: ['INV-0487.pdf', 'INV-0488.pdf', 'INV-0489.pdf', 'INV-0490.pdf', 'INV-0491.pdf', 'INV-0492.pdf', 'INV-0493.pdf', 'INV-0494.pdf', 'INV-0495.pdf', 'INV-0496.pdf', 'INV-0497.pdf', 'INV-0498.pdf', 'INV-0499.pdf', 'INV-0500.pdf', 'INV-0501.pdf', 'INV-0502.pdf', 'INV-0503.pdf', 'INV-0504.pdf', 'INV-0505.pdf', 'INV-0506.pdf', 'INV-0507.pdf', 'INV-0508.pdf', 'INV-0509.pdf', 'INV-0510.pdf', 'INV-0511.pdf', 'INV-0512.pdf', 'INV-0513.pdf', 'INV-0514.pdf', 'INV-0516.pdf', 'INV-0517.pdf', 'INV-0518.pdf', 'INV-0519.pdf', 'INV-0520.pdf']


In [4]:
# Reading a Pdf File
pdf_path = os.path.join(folder_path, pdf_files[0])
print("Reading PDF:", pdf_path)

Reading PDF: C:\Users\USER\Desktop\SQL and Excel Files\Test-20250930T093447Z-1-001\Test\Invoices\INV-0487.pdf


## Extracting one pdf file

In [5]:
# Displaying the pdf file
with pdfplumber.open(pdf_path) as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()

# Display first 1000 characters of the PDF
print(text[:1000])

Orbit Creative Solutions Ltd.
RC No: 402918
12 Orbit Crescent, Suntech District, Lagos, Nigeria
Email: accounts@orbitcreatives.ng
Phone: +234 801 234 5678
INVOICE
Invoice Number: INV-0487
Date of Issue: 25 Jul 2025
Due Date: 08 Aug 2025
Bill To:
Velocity AutoTech Ltd.
88 Redwood Lane, Central Bay
Lekki Peninsula, Lagos, Nigeria
Email: finance@velocityautotechltd.ng
Description Qty Unit Price (NGN) Total (NGN)
Digital Campaign Execution 1 3,278,034.00 3,278,034.00
Content Production & Editing 1 1,092,678.00 1,092,678.00
Micro-Influencer Activation 3 273,169.50 819,508.50
Campaign Oversight & Reporting 1 273,169.50 273,169.50
Subtotal: NGN 5,463,390.00
VAT (0%): NGN 0.00
TOTAL DUE: NGN 5,463,390.00
Payment Instructions
Bank: Unity National Bank
Account Name: Orbit Creative Solutions Ltd.
Account Number: 3019982221
Branch Code: 045229
Notes
- Payment is due within 14 days of the invoice date.
- Late payments are subject to a 2% monthly interest charge.
- Please use the invoice number as p

In [6]:
# Testing for Extraction
data = {
    'Invoice ID': None,
    'Client': None,
    'Email': None,
    'Start Date': None,
    'End Date': None,
    'Amount': None
}

In [7]:
# Extract Invoice ID
match = re.search(r"Invoice Number:\s*(\S+)", text)
if match:
    data['Invoice ID'] = match.group(1)

# Extract Dates
issue = re.search(r"Date of Issue:\s*([\d]{1,2}\s\w+\s\d{4})", text)
due = re.search(r"Due Date:\s*([\d]{1,2}\s\w+\s\d{4})", text)
if issue:
    data['Start Date'] = issue.group(1)
if due:
    data['End Date'] = due.group(1)

# Extract Client
client = re.search(r"Bill To:\s*([\w\s&]+Ltd\.)", text)
if client:
    data['Client'] = client.group(1).strip()

# Extract Client Email
email = re.search(r"Email:\s*([\w\.-]+@[\w\.-]+)", text)
if email:
    data['Email'] = email.group(1)

# Extract Total Amount
amount = re.search(r"TOTAL DUE:\s*NGN\s*([\d,]+\.\d{2})", text)
if amount:
    data['Amount'] = amount.group(1)

print(data)


{'Invoice ID': 'INV-0487', 'Client': 'Velocity AutoTech Ltd.', 'Email': 'accounts@orbitcreatives.ng', 'Start Date': '25 Jul 2025', 'End Date': '08 Aug 2025', 'Amount': '5,463,390.00'}


## Extracting all Files

In [8]:
# Store extracted data
records = []

# Loop through all PDF files
for file_name in os.listdir(folder_path):
    if file_name.lower().endswith(".pdf"):
        pdf_path = os.path.join(folder_path, file_name)

        data = {
            'Invoice ID': None,
            'Client': None,
            'Email': None,
            'Start Date': None,
            'End Date': None,
            'Amount': None
        }

        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:  # Sometimes extraction fails
                    text += page_text + "\n"

        # Extract Invoice ID
        match = re.search(r"Invoice Number:\s*(\S+)", text)
        if match:
            data['Invoice ID'] = match.group(1)

        # Extract Dates
        issue = re.search(r"Date of Issue:\s*([\d]{1,2}\s\w+\s\d{4})", text)
        due = re.search(r"Due Date:\s*([\d]{1,2}\s\w+\s\d{4})", text)
        if issue:
            data['Start Date'] = issue.group(1)
        if due:
            data['End Date'] = due.group(1)

        # Extract Client
        client = re.search(r"Bill To:\s*([^\n]+)", text)
        if client:
            data['Client'] = client.group(1).strip()

        # Extract Email (prefer last email → usually client’s)
        emails = re.findall(r"[\w\.-]+@[\w\.-]+", text)
        if emails:
            data['Email'] = emails[-1]

        # Extract Total Amount
        amount = re.search(r"TOTAL DUE:\s*NGN\s*([\d,]+\.\d{2})", text)
        if amount:
            data['Amount'] = amount.group(1)

        # Save record
        records.append(data)

# Convert to DataFrame
df = pd.DataFrame(records)

In [9]:
df.head()

,Invoice ID,Client,Email,Start Date,End Date,Amount
0,INV-0487,Velocity AutoTech Ltd.,finance@velocityautotechltd.ng,25 Jul 2025,08 Aug 2025,"5,463,390.00"
1,INV-0488,RapidTrak Logistics,finance@rapidtraklogistics.ng,26 Jul 2025,09 Aug 2025,"9,252,030.00"
2,INV-0489,NovaDrive Innovations,finance@novadriveinnovations.ng,02 Aug 2025,16 Aug 2025,"9,267,670.00"
3,INV-0490,RapidTrak Logistics,finance@rapidtraklogistics.ng,02 Aug 2025,16 Aug 2025,"6,328,045.00"
4,INV-0491,HexaConnect Services,finance@hexaconnectservices.ng,29 Jul 2025,12 Aug 2025,"8,384,366.00"


In [12]:
df.isnull().sum()

Invoice ID    0
Client        0
Email         0
Start Date    0
End Date      0
Amount        0
dtype: int64

## Saving my csv file

In [13]:
df.to_csv("invoices_extracted.csv", index=False)